In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from functools import partial

import optuna
from xfeat import SelectCategorical, LabelEncoder, Pipeline, ConcatCombination, SelectNumerical, ArithmeticCombinations, TargetEncoder, aggregation, GBDTFeatureSelector, GBDTFeatureExplorer

In [ ]:
# https://qiita.com/Ryo-Todaka/items/e97d6c065dcabe53dd08

In [2]:
df = pd.read_csv('reference/L01-2022P-2K.csv')

In [3]:
df.columns[87:]

Index(['属性移動Ｓ５９', '属性移動Ｓ６０', '属性移動Ｓ６１', '属性移動Ｓ６２', '属性移動Ｓ６３', '属性移動Ｈ１',
       '属性移動Ｈ２', '属性移動Ｈ３', '属性移動Ｈ４', '属性移動Ｈ５', '属性移動Ｈ６', '属性移動Ｈ７', '属性移動Ｈ８',
       '属性移動Ｈ９', '属性移動Ｈ１０', '属性移動Ｈ１１', '属性移動Ｈ１２', '属性移動Ｈ１３', '属性移動Ｈ１４',
       '属性移動Ｈ１５', '属性移動Ｈ１６', '属性移動Ｈ１７', '属性移動Ｈ１８', '属性移動Ｈ１９', '属性移動Ｈ２０',
       '属性移動Ｈ２１', '属性移動Ｈ２２', '属性移動Ｈ２３', '属性移動Ｈ２４', '属性移動Ｈ２５', '属性移動Ｈ２６',
       '属性移動Ｈ２７', '属性移動Ｈ２８', '属性移動Ｈ２９', '属性移動Ｈ３０', '属性移動Ｈ３１', '属性移動Ｒ２',
       '属性移動Ｒ３', '属性移動Ｒ４'],
      dtype='object')

In [4]:
dropcol = df.columns[87:]
df = df.drop(dropcol, axis=1)

In [5]:
df.columns

Index(['経度', '緯度', '所在地コード', '用途', '連番', '年次', '前年所在地コード', '前年用途', '前年連番',
       '市区町村名', '所在並びに地番', '住居表示', '行政', '地積', '利用の現況', '利用状況表示', '利用区分',
       '建物構造', '施設', '形状区分', '間口（比率）', '奥行（比率）', '階層（地上）', '階層（地下）', '前面道路区分',
       '前面道路の方位区分', '前面道路の幅員', '前面道路の駅前区分', '前面道路の舗装状況', '側道区分', '側道方位区分',
       '交通施設との近接区分', '周辺の土地の利用の現況', '駅名', '駅距離', '用途区分', '防火区分', '都市計画区分',
       '高度地区', '森林区分', '公園区分', '建蔽率', '容積率', '割増容積率', '共通地点区分', '対前年変動率',
       '選定年次ビット', 'Ｓ５８価格', 'Ｓ５９価格', 'Ｓ６０価格', 'Ｓ６１価格', 'Ｓ６２価格', 'Ｓ６３価格', 'Ｈ１価格',
       'Ｈ２価格', 'Ｈ３価格', 'Ｈ４価格', 'Ｈ５価格', 'Ｈ６価格', 'Ｈ７価格', 'Ｈ８価格', 'Ｈ９価格', 'Ｈ１０価格',
       'Ｈ１１価格', 'Ｈ１２価格', 'Ｈ１３価格', 'Ｈ１４価格', 'Ｈ１５価格', 'Ｈ１６価格', 'Ｈ１７価格', 'Ｈ１８価格',
       'Ｈ１９価格', 'Ｈ２０価格', 'Ｈ２１価格', 'Ｈ２２価格', 'Ｈ２３価格', 'Ｈ２４価格', 'Ｈ２５価格', 'Ｈ２６価格',
       'Ｈ２７価格', 'Ｈ２８価格', 'Ｈ２９価格', 'Ｈ３０価格', 'Ｈ３１価格', 'Ｒ２価格', 'Ｒ３価格', 'Ｒ４価格'],
      dtype='object')

In [6]:
dropcol = ['経度', '緯度', '用途', '連番', '年次', '前年所在地コード', '前年用途', '前年連番',
           '市区町村名', '所在並びに地番', '住居表示', '行政', '地積', '利用の現況', '利用状況表示', '利用区分',
           '建物構造', '施設', '形状区分', '間口（比率）', '奥行（比率）', '階層（地上）', '階層（地下）', '前面道路区分',
           '前面道路の方位区分', '前面道路の幅員', '前面道路の駅前区分', '前面道路の舗装状況', '側道区分', '側道方位区分',
           '交通施設との近接区分', '周辺の土地の利用の現況', '駅名', '駅距離', '用途区分', '防火区分', '都市計画区分',
           '高度地区', '森林区分', '公園区分', '建蔽率', '容積率', '割増容積率', '共通地点区分', '対前年変動率',
           '選定年次ビット',]
df = df.drop(dropcol, axis=1)

In [7]:
collist = ['Ｓ５８価格', 'Ｓ５９価格', 'Ｓ６０価格', 'Ｓ６１価格', 'Ｓ６２価格', 'Ｓ６３価格', 'Ｈ１価格',
       'Ｈ２価格', 'Ｈ３価格', 'Ｈ４価格', 'Ｈ５価格', 'Ｈ６価格', 'Ｈ７価格', 'Ｈ８価格', 'Ｈ９価格', 'Ｈ１０価格',
       'Ｈ１１価格', 'Ｈ１２価格', 'Ｈ１３価格', 'Ｈ１４価格', 'Ｈ１５価格', 'Ｈ１６価格', 'Ｈ１７価格', 'Ｈ１８価格',
       'Ｈ１９価格', 'Ｈ２０価格', 'Ｈ２１価格', 'Ｈ２２価格', 'Ｈ２３価格', 'Ｈ２４価格', 'Ｈ２５価格', 'Ｈ２６価格',
       'Ｈ２７価格', 'Ｈ２８価格', 'Ｈ２９価格', 'Ｈ３０価格', 'Ｈ３１価格', 'Ｒ２価格', 'Ｒ３価格', 'Ｒ４価格']

In [38]:
# 対象年度のデータを取得し、0以外のデータの平均値を取り、lastyearの平均価格が当時の何倍かを確認する。この倍率の常用対数を取り、当時の価格に加える。
correct = pd.DataFrame(index=[], columns=['所在地コード', '年度', '平均地価log', '地価_stdlog', '地価_maxlog', '地価_minlog', '地価_maxminlog', '地価_count'])
year = 1983
for colname in collist:
    price = df[['所在地コード', colname]][df[colname] != 0]
    price = price.groupby('所在地コード')
    table = price.mean()
    table = table.rename(columns={colname: '平均地価log'})
    table['平均地価log'] = pd.DataFrame(np.log10(table['平均地価log']))
    table['地価_stdlog'] = price.std()[colname]
    table.loc[table['地価_stdlog'].isna(),'地価_stdlog'] = 1
    table['地価_stdlog'] = pd.DataFrame(np.log10(table['地価_stdlog']))
    table['地価_maxlog'] = price.max()[colname]
    table['地価_maxlog'] = pd.DataFrame(np.log10(table['地価_maxlog']))
    table['地価_minlog'] = price.min()[colname]
    table['地価_minlog'] = pd.DataFrame(np.log10(table['地価_minlog']))
    table['地価_maxminlog'] = table['地価_maxlog'] - table['地価_minlog']
    table['地価_count'] = price.count()[colname]
    table['年度'] = year
    year += 1
    table = table.reset_index()
    correct = pd.concat([correct, table])
    
correct = correct.reset_index(drop=True)
correct

/usr/local/lib/python3.9/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.9/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.9/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.9/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.9/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.9/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: divide by zero encountered in log10
  result 

,所在地コード,年度,平均地価log,地価_stdlog,地価_maxlog,地価_minlog,地価_maxminlog,地価_count
0,1101,1983,5.742026,5.885713,6.380211,4.812913,1.567298,9
1,1102,1983,4.857709,4.936916,5.454845,4.361728,1.093117,8
2,1103,1983,4.836324,4.428763,5.079181,4.477121,0.60206,7
3,1104,1983,4.832509,4.178332,4.939519,4.732394,0.207125,4
4,1105,1983,4.831102,3.983841,4.897627,4.728354,0.169273,10
...,...,...,...,...,...,...,...,...
53924,47328,2022,5.052822,4.81275,5.20412,4.588832,0.615288,3
53925,47329,2022,4.929036,4.069128,5.0,4.822822,0.177178,8
53926,47348,2022,4.895975,4.362658,5.064458,4.716838,0.34762,6
53927,47350,2022,5.038024,4.50716,5.220108,4.906874,0.313235,6


In [39]:
correct.isnull().sum()

所在地コード          0
年度              0
平均地価log         0
地価_stdlog       0
地価_maxlog       0
地価_minlog       0
地価_maxminlog    0
地価_count        0
dtype: int64

In [40]:
# このデータをCSVとして出力する。
correct.to_csv('reference/correction.csv', index=False, header=True)